# Update Embedding Matrix

This notebook will highlight the process, result and insights obtained from allowing updates to the embedding matrix during training.

## Import Embedding Matrix

In [1]:
import json
from pathlib import Path

import numpy as np

embedding_path = Path("models/w2v_matrix.npy")
index_from_word_path = Path("models/w2v_index.json")

embedding_matrix = np.load(embedding_path)
with index_from_word_path.open() as f:
    index_from_word = json.load(f)

## Prepare Dataaset

(a) Import datasets

In [2]:
from utils.text import tokenize
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")
train_dataset = tokenize(dataset["train"])
val_dataset = tokenize(dataset["validation"])
test_dataset = tokenize(dataset["test"])

/Users/juinlee/Documents/GitHub/sc4002-nlp-sentiment-classification/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /Users/juinlee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     /Users/juinlee/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/juinlee/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
from utils.text import token_to_index

train_dataset = token_to_index(dataset=train_dataset, index_from_word=index_from_word)
val_dataset = token_to_index(dataset=val_dataset, index_from_word=index_from_word)
test_dataset = token_to_index(dataset=test_dataset, index_from_word=index_from_word)

train_dataset

Dataset({
    features: ['text', 'label', 'tokens', 'original_len', 'indexes'],
    num_rows: 8530
})

In [4]:
train_dataset = train_dataset.select_columns(["label", "original_len", "indexes"])
val_dataset = val_dataset.select_columns(["label", "original_len", "indexes"])
test_dataset = test_dataset.select_columns(["label", "original_len", "indexes"])

In [5]:
train_dataset.set_format(type="torch")
val_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

## Train RNN with Update to Embeddings

We use Optuna to perform heuristic search on optimal configuration when embeddings are updatabled during training.

In [6]:
import optuna
from utils.train import train_rnn_model_with_parameters

SEARCH_SPACE = {
    "batch_size": [32, 64, 128, 256, 512, 1024, 2048],
    "learning_rate": [1e-1, 1e-2, 1e-3, 1e-4],
    "optimizer_name": ["Adam"],
    # RNN Model Parameters
    "hidden_dim": [256, 128, 64, 32],
    "num_layers": [1, 2, 4],
    "sentence_representation_type": ["last", "average", "max"],
}

# Define the objective function for Optuna
def objective(trial):
    # Sample hyperparameters from search space
    hidden_dim = trial.suggest_categorical("hidden_dim", SEARCH_SPACE["hidden_dim"])
    num_layers = trial.suggest_int("num_layers", min(SEARCH_SPACE["num_layers"]), max(SEARCH_SPACE["num_layers"]))
    optimizer_name = trial.suggest_categorical("optimizer_name", SEARCH_SPACE["optimizer_name"])
    batch_size = trial.suggest_categorical("batch_size", SEARCH_SPACE["batch_size"])
    learning_rate = trial.suggest_categorical("learning_rate", SEARCH_SPACE["learning_rate"])
    sentence_representation_type = trial.suggest_categorical("sentence_representation_type", SEARCH_SPACE["sentence_representation_type"])
    
    log_message = f"---------- batch_size_{batch_size}; lr_{learning_rate}; optimizer_{optimizer_name}; hidden_dim_{hidden_dim}; num_layers_{num_layers}; sentence_representation_{sentence_representation_type} ----------"
    print(log_message)

    # Train the model and get the validation loss or accuracy
    val_acc = train_rnn_model_with_parameters(
        embedding_matrix=embedding_matrix,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        batch_size=batch_size,
        learning_rate=learning_rate,
        optimizer_name=optimizer_name,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        sentence_representation_type=sentence_representation_type,
        show_progress=True,
        freeze_embedding=False,
        log_dir="rnn/test/w2v"
    )
    
    # Return the metric to minimize (for example, validation loss)
    return val_acc

# Set up the Optuna study
study = optuna.create_study(direction="maximize")  # Change to "maximize" if you're maximizing an objective
study.optimize(objective, n_trials=50)  # Specify the number of trials you want

# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", best_params)


[I 2024-11-03 02:18:21,788] A new study created in memory with name: no-name-30756290-9226-488e-959b-e1cd0665e67f
Seed set to 42
/Users/juinlee/Documents/GitHub/sc4002-nlp-sentiment-classification/venv/lib/python3.9/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'rnn_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['rnn_model'])`.


---------- batch_size_512; lr_0.0001; optimizer_Adam; hidden_dim_32; num_layers_4; sentence_representation_average ----------


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/juinlee/Documents/GitHub/sc4002-nlp-sentiment-classification/venv/lib/python3.9/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 4.9 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
4.9 M     Trainable params
0         Non-trainable params
4.9 M     Total params
19.671    Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/juinlee/Documents/GitHub/sc4002-nlp-sentiment-classification/venv/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


...

Epoch 6: 100%|██████████| 9/9 [00:12<00:00,  0.73it/s, v_num=0, train_loss=0.104, train_acc=0.964, val_loss=0.754, val_acc=0.743]


[I 2024-11-03 03:22:44,955] Trial 31 finished with value: 0.7481059432029724 and parameters: {'hidden_dim': 128, 'num_layers': 2, 'optimizer_name': 'Adam', 'batch_size': 1024, 'learning_rate': 0.001, 'sentence_representation_type': 'max'}. Best is trial 25 with value: 0.7648007273674011.
Seed set to 42


---------- batch_size_1024; lr_0.001; optimizer_Adam; hidden_dim_32; num_layers_2; sentence_representation_last ----------


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 4.9 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
4.9 M     Trainable params
0         Non-trainable params
4.9 M     Total params
19.654    Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode

Detected KeyboardInterrupt, attempting graceful shutdown ...
[W 2024-11-03 03:23:02,843] Trial 32 failed with parameters: {'hidden_dim': 32, 'num_layers': 2, 'optimizer_name': 'Adam', 'batch_size': 1024, 'learning_rate': 0.001, 'sentence_representation_type': 'last'} because of the following error: NameError("name 'exit' is not defined").
Traceback (most recent call last):
  File "/Users/juinlee/Documents/GitHub/sc4002-nlp-senti

NameError: name 'exit' is not defined